In [ ]:
!pip install yfinance


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from scipy.optimize import minimize


tickers = ["VTSAX", "VTIAX", "VBTLX", "VDE", "VGSLX", "OPGSX", "BTC-USD"]
start_date = "2015-01-01"
end_date = "2023-01-01"

data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
daily_returns = data.pct_change().dropna()

num_portfolios = 5000
num_assets = len(tickers)

weights = np.zeros((num_portfolios, num_assets))

for i in range(num_portfolios):
    random_weights = np.random.random(num_assets)
    random_weights[4:6] = random_weights[4:6] * 0.20 / np.sum(random_weights[4:6])
    random_weights[6] = random_weights[6] * 0.10 / np.sum(random_weights[6])
    weights[i, :] = random_weights / np.sum(random_weights)

portfolio_returns = np.dot(daily_returns, weights.T)

volatility = np.std(portfolio_returns, axis=0)

annualized_volatility = volatility * np.sqrt(252)

sorted_volatilities = np.sort(annualized_volatility)
quartiles = np.percentile(sorted_volatilities, [25, 50, 75])

categories = ['Low', 'Medium', 'Moderate', 'High']
volatility_categories = pd.cut(annualized_volatility, bins=[0] + list(quartiles) + [np.inf], labels=categories, right=False)

volatility_data = pd.DataFrame({
    'Annualized Volatility': annualized_volatility,
    'Category': volatility_categories
})


[*********************100%%**********************]  7 of 7 completed


In [ ]:
X = weights
y_class = volatility_data['Category'].astype('category').cat.codes
y_reg = volatility_data['Annualized Volatility']

X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=None, max_features=1, random_state=42)
rf_classifier.fit(X_train, y_train_class)

rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=None, max_features=1, random_state=42)
rf_regressor.fit(X_train_reg, y_train_reg)

y_pred_class = rf_classifier.predict(X_test)
y_pred_reg = rf_regressor.predict(X_test_reg)

accuracy = accuracy_score(y_test_class, y_pred_class)
mse = mean_squared_error(y_test_reg, y_pred_reg)
r_squared = r2_score(y_test_reg, y_pred_reg)

print(f"Classifier Accuracy: {accuracy}")
print(f"Regressor Mean Squared Error: {mse}")
print(f"Regressor R-squared Value: {r_squared}")

Classifier Accuracy: 0.92
Regressor Mean Squared Error: 7.751834721707441e-05
Regressor R-squared Value: 0.9652789905605766


In [ ]:
expected_returns = np.array([0.05, 0.10, 0.12, 0.07, 0.03, 0.09, 0.11])
covariance_matrix = np.random.rand(7, 7)
covariance_matrix = (covariance_matrix + covariance_matrix.T) / 2
np.fill_diagonal(covariance_matrix, 0.1)


risk_free_rate = 0.02 # risk free rate

def objective(weights):
    portfolio_return = np.sum(weights * expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
    return -sharpe_ratio

constraints = (
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
    {'type': 'ineq', 'fun': lambda x: x - 0.2}
)

bounds = tuple((0, 1) for asset in range(len(expected_returns)))
initial_weights = np.array(len(expected_returns) * [1. / len(expected_returns)])
result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)
optimized_weights = result.x
print("Optimized Weights:", optimized_weights)
print("Expected Portfolio Return:", np.sum(optimized_weights * expected_returns))
print("Expected Portfolio Volatility:", np.sqrt(np.dot(optimized_weights.T, np.dot(covariance_matrix, optimized_weights))))
print("Sharpe Ratio:", (np.sum(optimized_weights * expected_returns) - risk_free_rate) / np.sqrt(np.dot(optimized_weights.T, np.dot(covariance_matrix, optimized_weights))))


Optimized Weights: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.14285714
 0.14285714]
Expected Portfolio Return: 0.08142857142873532
Expected Portfolio Volatility: 0.6517280821266038
Sharpe Ratio: 0.09425490954493239
